In [19]:
import os, glob
import numpy as np
import pandas as pd
import math
import pickle
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import stats
from statsmodels.stats.weightstats import ztest
from IPython.display import display, HTML

In [26]:
non_coding_region= "donor"
cancer_type= "Brain"

In [21]:
base_dir = "/data/private/pdutta/DNABERT_data/Genetic_variant/Brain/{}".format(non_coding_region)

In [22]:
df_donor = pd.read_csv("/data/private/pdutta/DNABERT_data/Splice_sites/Donor_raw_files/Unique_Donor_RAW_Combined_BED.tsv", sep="\t")
df_donor

,chr_name,start,end,strand,transcript_id,exon_id,transcript_type
0,chr1,12189,12268,+,"ENST00000450305.2, ENST00000456328.2","ENSE00001671638.2, ENSE00002234944.1","transcribed_unprocessed_pseudogene, processed_..."
1,chr1,12659,12738,+,ENST00000450305.2,ENSE00001758273.2,transcribed_unprocessed_pseudogene
2,chr1,12683,12762,+,ENST00000456328.2,ENSE00003582793.1,processed_transcript
3,chr1,13336,13415,+,ENST00000450305.2,ENSE00001746346.2,transcribed_unprocessed_pseudogene
4,chr1,15756,15835,-,ENST00000488147.1,ENSE00002030414.1,unprocessed_pseudogene
...,...,...,...,...,...,...,...
297951,chrY,57211531,57211610,+,ENST00000464205.6_PAR_Y,ENSE00003785071.1,processed_transcript
297952,chrY,57211582,57211661,+,"ENST00000340131.12_PAR_Y, ENST00000359512.8_PA...","ENSE00003627301.1, ENSE00003905275.1, ENSE0000...","retained_intron, protein_coding"
297953,chrY,57213164,57213243,-,ENST00000507418.6_PAR_Y,ENSE00002036959.1,unprocessed_pseudogene
297954,chrY,57213840,57213919,-,ENST00000507418.6_PAR_Y,ENSE00002046926.1,unprocessed_pseudogene


In [27]:
with open("/data/projects/GDC_Cancer_Wise/{}/Data/Sequence_generation/raw_{}_vcf_data.pkl".format(cancer_type, non_coding_region), "rb") as file:
    loaded_dictionary = pickle.load(file)

In [13]:
# with open("/data/projects/GDC_Cancer_Wise/Brain/Data/Sequence_generation/first_5_pairs.pkl", "rb") as file:
#     loaded_dictionary = pickle.load(file)

In [28]:
first_element_key, first_element_value = next(iter(loaded_dictionary.items()))

In [29]:
first_element_key

'b1e62d8e-24d2-4118-8cd0-3142acebdd5b'

In [30]:
first_element_value

,index,chr,strand,Transcript_ID,Acceptor_start,Acceptor_end,varinat_start,variant_end,ref_neucleotide,alternative_neucleotide,reference_seq,alt_seq
0,0,chr1,+,ENST00000642116.1,58818,58897,58865,58866,C,G,GCTCCTTTGGGTCATCAGGATTTCTTCAACAATAAAATGTAATAAT...,GCTCCTTTGGGTCATCAGGATTTCTTCAACAATAAAATGTAATAAT...
1,1,chr1,-,ENST00000623083.4,187089,187168,187101,187102,C,G,CACACCAGAAATCAGGTGGCCTCAAAGAGCTGCTCCCACCTGAAGG...,CACACCAGAAATGAGGTGGCCTCAAAGAGCTGCTCCCACCTGAAGG...
2,2,chr1,-,ENST00000623083.4,187715,187794,187743,187744,A,G,CCACTGCACCTGTCTCCACCTCTGTCCCACCCCTCCCACCTGCTGT...,CCACTGCACCTGTCTCCACCTCTGTCCCGCCCCTCCCACCTGCTGT...
3,3,chr1,-,"ENST00000417636.2, ENST00000634833.2",586781,586860,586843,586844,G,A,AACCTTGCAAATAGATTTCTTGAAATGATTGACACTCACCTTGGTT...,AACCTTGCAAATAGATTTCTTGAAATGATTGACACTCACCTTGGTT...
4,4,chr1,-,ENST00000634833.2,594589,594668,594620,594621,A,G,TGGAATATCCTTAAGATAAGGCCCCAGAACTAAAAGTACCCTGTCA...,TGGAATATCCTTAAGATAAGGCCCCAGAACTGAAAGTACCCTGTCA...
...,...,...,...,...,...,...,...,...,...,...,...,...
25328,25328,chrY,-,ENST00000331172.7,11382830,11382909,11382840,11382841,T,C,TTTGAAATAATCTTACTCGATTTGTAATTGTAAATCTACCTGTGGA...,TTTGAAATAACCTTACTCGATTTGTAATTGTAAATCTACCTGTGGA...
25329,25329,chrY,-,ENST00000331172.7,11385477,11385556,11385522,11385523,A,G,AGTCAATAATAAAGAAAAGAGAGCTAATTATATACTTACCAAGAAA...,AGTCAATAATAAAGAAAAGAGAGCTAATTATATACTTACCAAGAAG...
25330,25330,chrY,-,ENST00000331172.7,11385477,11385556,11385555,11385556,A,T,AGTCAATAATAAAGAAAAGAGAGCTAATTATATACTTACCAAGAAA...,AGTCAATAATAAAGAAAAGAGAGCTAATTATATACTTACCAAGAAA...
25331,25331,chrY,+,ENST00000614446.1,11417439,11417518,11417475,11417476,C,A,AAAATGAATCCACAGCTCTATCTCAGGTTCTAGCCCCGGTATTCGT...,AAAATGAATCCACAGCTCTATCTCAGGTTCTAGCCCAGGTATTCGT...


In [31]:
def log_oods_ratio(p1,p2):
    term1 = math.log2(p1/(1-p1))
    term2 = math.log2(p2/(1-p2))
    return (term1-term2)
def calculate_diff_probab(p1,p2):
    return((p2-p1)*max(p1,p2))

In [41]:
patient_dfs= []
patient_id = []
for dirpath, dirnames, filenames in os.walk(base_dir):
    #print(dirpath, dirnames, filenames)
    for dir_name in dirnames:
        dir_path = dirpath +"/" + dir_name
        print(dir_path)
        patient_id.append(dir_name)
        df_temp = loaded_dictionary[dir_name]
        concat_probab = np.load(dir_path+"/Prediction_result/pred_results.npy")
        print(len(concat_probab))
        Reference_probab = concat_probab[::2]
        Alternative_probab = concat_probab[1::2]
        df_temp['Ref_probab']=Reference_probab
        df_temp['Alt_probab']=Alternative_probab
        df_temp['ScoreChange'] = df_temp.apply(lambda row: calculate_diff_probab(row['Ref_probab'], row['Alt_probab']), axis=1)
        df_temp['LogOddRatio'] = df_temp.apply(lambda row: log_oods_ratio(row['Ref_probab'], row['Alt_probab']), axis=1)
        #display(df_temp)
        df_temp.rename(columns={'Acceptor_start': 'donor_start', 'Acceptor_end': 'donor_end'}, inplace=True)
        patient_dfs.append(df_temp)
        #input()
    break

/data/private/pdutta/DNABERT_data/Genetic_variant/Brain/donor/722d3df5-7254-44f0-bbe0-cf39dbd57866
57926
/data/private/pdutta/DNABERT_data/Genetic_variant/Brain/donor/ddd29528-58ad-461f-8509-4e42f71b49fb
30024
/data/private/pdutta/DNABERT_data/Genetic_variant/Brain/donor/9593d4de-0dab-4d1e-b86c-af91cac938e6
40062
/data/private/pdutta/DNABERT_data/Genetic_variant/Brain/donor/e849abef-b55e-4124-b760-09101a7c4390
52996
/data/private/pdutta/DNABERT_data/Genetic_variant/Brain/donor/0c700657-b0b8-4361-9f89-b89a6c7b262d
47666
/data/private/pdutta/DNABERT_data/Genetic_variant/Brain/donor/a60a9995-6a02-4035-9c3e-2fee0e6e87c5
39128
/data/private/pdutta/DNABERT_data/Genetic_variant/Brain/donor/4bc99b08-8b67-443e-88a4-03edef39d433
52388
/data/private/pdutta/DNABERT_data/Genetic_variant/Brain/donor/0fce50b4-028d-4b54-aa49-3a8e28a7f7c6
52914
/data/private/pdutta/DNABERT_data/Genetic_variant/Brain/donor/8f36ad24-c425-4474-9d03-a5198e2e4901
51788
/data/private/pdutta/DNABERT_data/Genetic_variant/Brain

In [43]:
# Concatenate the patient dataframes and drop duplicates to get all unique regions across the patients
unique_patient_regions = pd.concat(patient_dfs).drop_duplicates(subset=['chr', 'Transcript_ID', 'donor_start', 'donor_end'])
unique_patient_regions

,index,chr,strand,Transcript_ID,donor_start,donor_end,varinat_start,variant_end,ref_neucleotide,alternative_neucleotide,reference_seq,alt_seq,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
0,0,chr1,-,ENST00000488147.1,29494,29573,29570,29571,G,C,CCTGTCGGGCCGAGTCGCGGGCCTGGGCACGGAACTCACGCTCACT...,CCTGTCGGGCCGAGTCGCGGGCCTGGGCACGGAACTCACGCTCACT...,0.581049,0.548680,-0.018808,0.190062
1,1,chr1,-,ENST00000623083.4,186277,186356,186290,186291,G,A,CCCAAGGCCCTCCGACTGCAAGCTCCAGGGCCCGCTCACCTTGCTC...,CCCAAGGCCCTCCAACTGCAAGCTCCAGGGCCCGCTCACCTTGCTC...,0.229444,0.142084,-0.020044,0.846328
3,3,chr1,-,ENST00000623083.4,187089,187168,187101,187102,C,G,CACACCAGAAATCAGGTGGCCTCAAAGAGCTGCTCCCACCTGAAGG...,CACACCAGAAATGAGGTGGCCTCAAAGAGCTGCTCCCACCTGAAGG...,0.983258,0.982440,-0.000804,0.069987
4,4,chr1,-,ENST00000623083.4,187715,187794,187743,187744,A,G,CCACTGCACCTGTCTCCACCTCTGTCCCACCCCTCCCACCTGCTGT...,CCACTGCACCTGTCTCCACCTCTGTCCCGCCCCTCCCACCTGCTGT...,0.126169,0.193841,0.013118,-0.735804
5,5,chr1,-,ENST00000442116.1,264564,264643,264626,264627,A,G,CTCAGCACACGGGGAGCAGACAGGAAGCTGTTGCCTCACCTTCCTC...,CTCAGCACACGGGGAGCAGACAGGAAGCTGTTGCCTCACCTTCCTC...,0.015331,0.016494,0.000019,-0.107159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9331,9331,chr16,-,"ENST00000330048.9, ENST00000431994.5",32879578,32879657,32879579,32879580,G,A,CGCCACCATGCCCCAGCCCAGCCCTCAGCCATGACCTACCATACAC...,CACCACCATGCCCCAGCCCAGCCCTCAGCCATGACCTACCATACAC...,0.123305,0.091909,-0.003871,0.474716
10077,10077,chr17,-,"ENST00000225696.8, ENST00000572019.1, ENST0000...",5419314,5419393,5419342,5419343,C,T,TCCCGGTGAGGGTCACTTCCAAGATCGGCCTGGCATTACCTGGTAC...,TCCCGGTGAGGGTCACTTCCAAGATCGGTCTGGCATTACCTGGTAC...,0.444759,0.348589,-0.042772,0.581949
23406,23406,chr7,+,ENST00000441928.1,113107766,113107845,113107831,113107832,G,T,TTGGAAGAGTGAAAACAGTTTTGCTGAGGACAGGTAAGGTATGGGT...,TTGGAAGAGTGAAAACAGTTTTGCTGAGGACAGGTAAGGTATGGGT...,0.122750,0.122561,-0.000023,0.002538
25712,25712,chr9,+,ENST00000318560.6,130835487,130835566,130835507,130835508,C,T,GAAGGGGCTGTCCTCGTCCTCCAGCTGTTATCTGGAAGGTAAGCCC...,GAAGGGGCTGTCCTCGTCCTTCAGCTGTTATCTGGAAGGTAAGCCC...,0.419747,0.341496,-0.032846,0.480162


In [44]:
unique_patient_regions[unique_patient_regions['donor_start']==28452437]

,index,chr,strand,Transcript_ID,donor_start,donor_end,varinat_start,variant_end,ref_neucleotide,alternative_neucleotide,reference_seq,alt_seq,Ref_probab,Alt_probab,ScoreChange,LogOddRatio


In [ ]:
# Step 3: Create a new dataframe with three rows for each patient, with columns as the acceptor coordinates

# Initialize the dictionaries
data_ref, data_alt, data_log_odd, data_score_change = {}, {}, {},  {}

# Loop over each region in the unique_patient_regions
for _, region in unique_patient_regions.iterrows():
    key = f'{region.chr}_{region["{}_start".format(non_coding_region)]}_{region["{}_end".format(non_coding_region)]}_{region.varinat_start}_{region.variant_end}'
    data_ref[key] = []
    data_alt[key] = []
    data_log_odd[key] = []
    data_score_change[key] = []

# Loop through each patient
for df in patient_dfs:
    print(df.shape)
    for _, region in unique_patient_regions.iterrows():
        for data, score_column in zip([data_ref, data_alt, data_log_odd, data_score_change], ['Ref_probab', 'Alt_probab', 'LogOddRatio', 'ScoreChange']):
            score_series = df.loc[(df['chr'] == region.chr) & (df['{}_start'.format(non_coding_region)] == region["{}_start".format(non_coding_region)]) & (df['varinat_start'] == region.varinat_start), score_column]
            if score_series.empty:
                score = np.nan
            else:
                score = score_series.max()
            data[f'{region.chr}_{region["{}_start".format(non_coding_region)]}_{region["{}_end".format(non_coding_region)]}_{region.varinat_start}_{region.variant_end}'].append(score)
    print(df)

(28963, 16)
       index   chr strand      Transcript_ID  donor_start  donor_end  \
0          0  chr1      -  ENST00000488147.1        29494      29573   
1          1  chr1      -  ENST00000623083.4       186277     186356   
2          2  chr1      -  ENST00000623083.4       186277     186356   
3          3  chr1      -  ENST00000623083.4       187089     187168   
4          4  chr1      -  ENST00000623083.4       187715     187794   
...      ...   ...    ...                ...          ...        ...   
28958  28958  chrY      -  ENST00000331172.7     11381588   11381667   
28959  28959  chrY      -  ENST00000331172.7     11382830   11382909   
28960  28960  chrY      -  ENST00000331172.7     11385477   11385556   
28961  28961  chrY      -  ENST00000331172.7     11385477   11385556   
28962  28962  chrY      +  ENST00000614446.1     11417439   11417518   

       varinat_start  variant_end ref_neucleotide alternative_neucleotide  \
0              29570        29571             

In [ ]:
result_df_ref = pd.DataFrame(data_ref)
result_df_ref.insert(0, "Patient_ID", patient_id)
result_df_ref

In [ ]:
result_df_alt = pd.DataFrame(data_alt)
result_df_alt.insert(0, "Patient_ID", patient_id)
result_df_alt

In [ ]:
result_df_log_odd = pd.DataFrame(data_log_odd)
result_df_log_odd.insert(0, "Patient_ID", patient_id)
result_df_log_odd

In [ ]:
result_df_score_change = pd.DataFrame(data_score_change)
result_df_score_change.insert(0, "Patient_ID", patient_id)
result_df_score_change

In [ ]:
result_df_ref.to_csv("/data/projects/GDC_Cancer_Wise/Brain/Data/DNABERT_result/{}/df_ref_score.tsv".format(non_coding_region), sep="\t", index=False)

In [ ]:
result_df_alt.to_csv("/data/projects/GDC_Cancer_Wise/Brain/Data/DNABERT_result/{}/df_alt_score.tsv".format(non_coding_region), sep="\t", index=False)

In [ ]:
result_df_log_odd.to_csv("/data/projects/GDC_Cancer_Wise/Brain/Data/DNABERT_result/{}/df_log_odd_score.tsv", sep="\t".format(non_coding_region), index=False)

In [ ]:
result_df_score_change.to_csv("/data/projects/GDC_Cancer_Wise/Brain/Data/DNABERT_result/{}/df_log_odd_score.tsv", sep="\t".format(non_coding_region), index=False)